In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/multi_turn_rewrite/chinese/main')

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
import time
import numpy as np

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.2
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def align_pad(li):
  max_len = max([len(sent) for sent in li])
  for sent in li:
    if len(sent) < max_len:
      sent += [0] * (max_len - len(sent))

In [0]:
def parse_fn(text):
  h1, h2, q = text.rstrip().split('|')
  char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
  h1, h2, q = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q)
  align_pad([h1, h2])
  return {'query': np.reshape(q, (1, len(q))),
          'history':np.reshape([h1, h2], (1, 2, len(h1)))
  }

In [6]:
params = {'export_dir': '../model/baseline_lstm_beam_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])
print(latest)

predict_fn = tf.contrib.predictor.from_saved_model(latest)

../model/baseline_lstm_beam_export/1587959561
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/baseline_lstm_beam_export/1587959561/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  print('A:', ' '.join([params['idx2char'].get(idx, '<unk>') for idx in predictions['output'][0] if (idx != 0 and idx != 2)]))
  print('Cost %.2f sec' % (time.time() - t0))
  print()

Input:iphonex 好不好|iphone不好用|为什么不好用
A: i p h o n e x 什 么 不 好 用
Cost 1.03 sec

Input:抄的菜不香|我想炒一个鱼香肉丝|不好吃
A: 鱼 香 肉 丝 不 好 吃
Cost 0.11 sec

Input:我要玩守望先锋|可以玩的没事的|你送我个账号吧
A: 守 望 先 锋 送 我 个 账 号 吧
Cost 0.12 sec

Input:上海啊|上海今天的天气是多云转晴26度到32度东北风小于3级|很好的天气
A: 上 海 很 好 的 天 气
Cost 0.16 sec

Input:对|这段时间新流星花园挺火的你看了吗|讲了什么
A: 新 流 星 花 园 讲 了 什 么
Cost 0.14 sec

Input:你认识杨幂吗|杨幂得罪的人不是很多吗|她长什么样
A: 杨 幂 长 什 么 样
Cost 0.11 sec

Input:你可以去搜一下天津之眼的图片|风景好图片好|评价一下
A: 评 价 一 下 天 津 之 眼 的 图 片
Cost 0.14 sec

Input:你喜欢偶像练习生的谁|看完get了好多帅气的小哥哥害羞|都有谁
A: 偶 像 练 习 生 都 有 谁
Cost 0.15 sec

Input:tfboys是谁你都不知道|明星呗|是很出名的明星也是最火的明星
A: t f b o y s 是 很 出 名 的 明 星 也 是 最 火 的 明 星
Cost 0.24 sec

Input:好羡慕玖月奇迹要是俩人能结婚就好了愉快|结婚啦|喜欢这个声音
A: 喜 欢 慕 玖 玖 月 奇 迹
Cost 0.15 sec

Input:鞠婧祎的照片|哈哈 去看照片|照片
A: 鞠 婧 祎 的 照 片
Cost 0.08 sec

Input:你知道孔子吗|创立儒家学派的孔老先生真是个很伟大的人|他有什么著作
A: 孔 子 有 什 么 著 作
Cost 0.16 sec

Input:不不不刘亮程是个作家|微小说|他写过一个人的村庄
A: 微 小 说 写 过 一 个 人 的 村 庄
Cost 0.14 sec

Input:你读过阿西莫夫吗|艾西莫夫基地系列一共7本|你读过吗
A: 你 读 过 阿 西 莫 夫 吗
Cost 0.13 sec

Input:你喜欢五月天吗|超级喜欢